<a href="https://colab.research.google.com/github/breyer/tat/blob/main/TAT_EMA520_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data is based on the following backtest for the period from 1/1/2023 to 6/16/2023:**

https://tradeautomationtoolbox.com/byob-ticks/?save=eLwvpkg

I would love to get feedback ...



In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
from datetime import datetime

!rm *.png *.html *.zip


def filter_data(df, days):
    end_date = df['EntryTime'].max()
    start_date = end_date - pd.DateOffset(days=days)
    return df.loc[(df['EntryTime'] >= start_date) & (df['EntryTime'] <= end_date)].copy()

def rank_trading_days(df):
    profit_per_day = df.groupby('EntryDayOfWeek')['ProfitLossAfterSlippage'].sum()*100
    return profit_per_day.sort_values(ascending=False)

def rank_trading_times(df):
    df['EntryHourMinute'] = df['EntryTime'].dt.strftime('%H:%M')
    profit_per_time = df.groupby('EntryHourMinute')['ProfitLossAfterSlippage'].sum()*100
    return profit_per_time.sort_values(ascending=False)
from datetime import datetime

def top_trading_times(df, n=3):
    df['EntryHourMinute'] = df['EntryTime'].dt.strftime('%H:%M')
    profit_per_time = df.groupby('EntryHourMinute')['ProfitLossAfterSlippage'].sum()*100
    top_times = profit_per_time.nlargest(n).reset_index()  # Get a DataFrame with EntryHourMinute as a column
    top_times['EntryHourMinute'] = top_times['EntryHourMinute'].apply(lambda x: datetime.strptime(x, '%H:%M'))  # Convert to datetime for sorting
    top_times = top_times.sort_values('EntryHourMinute')  # Sort by time
    time_str = ' ; '.join(top_times['EntryHourMinute'].dt.strftime('%H:%M'))  # Convert back to string
    return time_str

def plot_heatmap(df, title, filename):
    df['EntryHour'] = df['EntryTime'].dt.hour
    df['EntryMinute'] = df['EntryTime'].dt.minute
    pivot_table = df.pivot_table(index='EntryHour', columns='EntryMinute', values='ProfitLossAfterSlippage', aggfunc='sum', fill_value=0)

    colors = sns.diverging_palette(10, 130, n=256).as_hex()
    colors = ['black'] + colors
    cmap = ListedColormap(colors)

    plt.figure(figsize=(10, 10))
    sns.heatmap(pivot_table, cmap=cmap, center=0, cbar=False)
    plt.title(title)
    plt.xlabel('Minute')
    plt.ylabel('Hour')
    plt.savefig(filename, format='png')
    plt.close()

def plot_heatmap_all_data(df, filename):
    df['EntryHour'] = df['EntryTime'].dt.hour
    df['EntryMinute'] = df['EntryTime'].dt.minute
    pivot_table = df.pivot_table(index='EntryHour', columns='EntryMinute', values='ProfitLossAfterSlippage', aggfunc='sum', fill_value=0)

    colors = sns.diverging_palette(10, 130, n=256).as_hex()
    colors = ['black'] + colors
    cmap = ListedColormap(colors)

    plt.figure(figsize=(5, 5))
    sns.heatmap(pivot_table, cmap=cmap, center=0)
    plt.title('Profit and Loss per Entry Time (Hour and Minute) - All Data')
    plt.xlabel('Minute')
    plt.ylabel('Hour')
    plt.savefig(filename, format='png')
    plt.close()

df = pd.read_csv("Trades.csv")
df['EntryTime'] = pd.to_datetime(df['EntryTime'])
df['EntryDate'] = df['EntryTime'].dt.date

days_list = [90, 60, 45, 30, 20, 15, 10, 5]

# Start of HTML content and Table of Contents
html_content = """
<h1>Table of Contents</h1>
<ol>
    <li><a href="#all_data">Best times to enter a trade for all data</a></li>
    <li><a href="#profit_per_day">Profit per day of the week</a></li>
"""

for days in days_list:
    html_content += f"<li><a href='#heatmap_{days}_days'>Best times to enter a trade for the last {days} days</a></li>\n"

html_content += "</ol>"

# Actual content
for days in days_list:
    df_filtered = filter_data(df, days)
    filename = f'heatmap_{days}_days.png'
    plot_heatmap(df_filtered, f'Profit and Loss per Entry Time (Hour and Minute) - Last {days} Days', filename)
    html_content += f"<h3 id='heatmap_{days}_days'>Best times to enter a trade for the last {days} days:</h3>\n"
    html_content += f"<img src='{filename}' alt='Heatmap for {days} days'><br>\n"
    html_content += f"<table>{rank_trading_times(df_filtered).to_frame().reset_index().to_html(index=False)}</table>\n"
    html_content += f"<h3>Top {11} times to enter a trade:</h3>\n"
    html_content += f"<p>{top_trading_times(df_filtered, 11)}</p>\n"

filename = 'heatmap_all_data.png'
plot_heatmap_all_data(df, filename)
html_content += "<h3 id='all_data'>Best times to enter a trade for all data:</h3>\n"
html_content += f"<img src='{filename}' alt='Heatmap for all data'><br>\n"
html_content += f"<table>{rank_trading_times(df).to_frame().reset_index().to_html(index=False)}</table>\n"

df['EntryDayOfWeek'] = df['EntryTime'].dt.dayofweek
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df['EntryDayOfWeek'] = df['EntryDayOfWeek'].apply(lambda x: days_of_week[x])

html_content += "<h3 id='profit_per_day'>Profit per day of the week:</h3>\n"
html_content += f"<p>{rank_trading_days(df)}</p>\n"


rm: cannot remove '*.html': No such file or directory
rm: cannot remove '*.zip': No such file or directory


In [27]:
pivot_table = df.pivot_table(index='EntryDate', columns='EntryHourMinute', values='ProfitLossAfterSlippage', aggfunc='sum', fill_value=0)

colors = sns.diverging_palette(10, 130, n=256).as_hex()
colors = ['black'] + colors
cmap = ListedColormap(colors)

plt.figure(figsize=(20, 20))
sns.heatmap(pivot_table, cmap=cmap, center=0)
plt.title('Profit and Loss per Entry Time (Hour and Minute)')
plt.xlabel('Time')
plt.ylabel('Date')
plt.savefig("heatmap_date_time.png")
plt.close()

html_content += f"<h3>Heatmap of Profit and Loss per Entry Date and Time:</h3>\n"
html_content += f"<img src='heatmap_date_time.png' alt='Heatmap of Profit and Loss per Entry Date and Time'><br>\n"

In [28]:
pivot_table = df.pivot_table(index='EntryDayOfWeek', columns='EntryHourMinute', values='ProfitLossAfterSlippage', aggfunc='sum', fill_value=0)

colors = sns.diverging_palette(10, 130, n=256).as_hex()
colors = ['black'] + colors
cmap = ListedColormap(colors)

plt.figure(figsize=(20, 20))
sns.heatmap(pivot_table, cmap=cmap, center=0)
plt.title('Profit and Loss per Entry Time (Hour and Minute)')
plt.xlabel('Time')
plt.ylabel('Day of Week')
plt.savefig("heatmap_dayofweek_time.png")
plt.close()

html_content += f"<h3>Heatmap of Profit and Loss per Entry Day of Week and Time:</h3>\n"
html_content += f"<img src='heatmap_dayofweek_time.png' alt='Heatmap of Profit and Loss per Entry Day of Week and Time'><br>\n"

with open('heatmap_report.html', 'w') as f:
    f.write(html_content)

!zip archive.zip *.png *.html

  adding: heatmap_10_days.png (deflated 33%)
  adding: heatmap_15_days.png (deflated 33%)
  adding: heatmap_20_days.png (deflated 32%)
  adding: heatmap_30_days.png (deflated 33%)
  adding: heatmap_45_days.png (deflated 34%)
  adding: heatmap_5_days.png (deflated 33%)
  adding: heatmap_60_days.png (deflated 34%)
  adding: heatmap_90_days.png (deflated 33%)
  adding: heatmap_all_data.png (deflated 14%)
  adding: heatmap_date_time.png (deflated 32%)
  adding: heatmap_dayofweek_time.png (deflated 44%)
  adding: heatmap_report.html (deflated 89%)
